In [17]:
from youtube_transcript_api import YouTubeTranscriptApi
from nltk.tokenize import TextTilingTokenizer 
from youtubesearchpython import VideosSearch
from semantic_search import SemanticSearch 
import pandas as pd
import gradio as gr
import numpy as np
import requests
import tiktoken
import openai
import json
import nltk
import re
import os

In [18]:
def set_openai_key(key):
    if key == "env":
        key = os.environ.get("OPENAI_API_KEY")
    openai.api_key = key

def get_youtube_data(url):

    video_id = url.split("=")[1]

    try:
        raw = YouTubeTranscriptApi.get_transcript(video_id)
    except:
        try:
            transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
            for transcript in transcript_list:
                raw = transcript.translate('en').fetch()
                break
        except:
            print(f"No transcript found for {url}") # Usually because the video itself disabled captions
            return False

    response = requests.get(f"https://noembed.com/embed?dataType=json&url={url}")
    data = json.loads(response.content)

    title, author = data["title"], data["author_name"]

    # Remove any "'" from title
    title = title.replace("'", "")
    author = author.replace("'", "")

    df = pd.DataFrame(raw)

    df['end'] = df['start'] + df['duration']
    df['total_words'] = df['text'].apply(lambda x: len(x.split())).cumsum()
    df["text"] = df["text"] + "\n\n"

    return df, title, author

def to_timestamp(seconds):
    seconds = int(seconds)

    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds_remaining = seconds % 60
    
    if seconds >= 3600:
        return f"{hours:02d}:{minutes:02d}:{seconds_remaining:02d}"
    else:
        return f"{minutes:02d}:{seconds_remaining:02d}"

def to_seconds(timestamp):
    time_list = timestamp.split(':')
    total_seconds = 0
    if len(time_list) == 2:  # Minutes:Seconds format
        total_seconds = int(time_list[0]) * 60 + int(time_list[1])
    elif len(time_list) == 3:  # Hours:Minutes:Seconds format
        total_seconds = int(time_list[0]) * 3600 + int(time_list[1]) * 60 + int(time_list[2])
    else:
        raise ValueError("Invalid timestamp format")
    return total_seconds

def get_segments(df, title, author, split_by_topic, segment_length = 200):

    transcript = df['text'].str.cat(sep=' ')

    if not split_by_topic:
        words = transcript.split()
        segments = [' '.join(words[i:i+segment_length]) for i in range(0, len(words), segment_length)]
    else:
        try:
            segments = tt.tokenize(transcript)
        except:
            return ""

    segments = [segment.replace('\n','').strip() for segment in segments]

    segments_wc = [len(segment.split()) for segment in segments]
    segments_wc = np.cumsum(segments_wc)

    idx = [np.argmin(np.abs(df['total_words'] - total_words)) for total_words in segments_wc]

    segments_end_times = df['end'].iloc[idx].values
    segments_end_times = np.insert(segments_end_times, 0, 0.0)

    segments_times = [f"({to_timestamp(segments_end_times[i-1])}, {to_timestamp(segments_end_times[i])})" for i in range(1,len(segments_end_times))]

    segments_text = [f"Segment from '{title}' by {author}\nTimestamp: {segment_time}\n\n{segment}\n" for segment, segment_time in zip(segments, segments_times)]

    return segments_text

def fit_searcher(segments, n_neighbours):
    global searcher
    searcher.fit(segments, n_neighbors=n_neighbours)
    return True

def num_tokens(text, model):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def refencify(text):
    title_pattern = r"Segment from '(.+)'"
    timestamp_pattern = r"Timestamp: \((.+)\)"

    print(text)

    title = re.search(title_pattern, text).group(1)
    timestamp = re.search(timestamp_pattern, text).group(1).split(",")
    start_timestamp, end_timestamp = timestamp

    url = titles_to_urls[title]
    start_seconds = to_seconds(start_timestamp)
    end_seconds = to_seconds(end_timestamp)

    video_iframe = f'''<iframe
    width="400"
    height="240"
    src="{url.replace("watch?v=", "embed/")}?start={start_seconds}&end={end_seconds}&controls=0"
    frameborder="0"
    allow="accelerometer; autoplay; modestbranding; encrypted-media; gyroscope; picture-in-picture"
    allowfullscreen
    >
    </iframe>'''

    return start_timestamp, end_timestamp, f"{video_iframe}\n\n"

def form_query(question, model, token_budget):

    results = searcher(question)

    introduction = 'Use the below segments from multiple youtube videos to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer." Cite each sentence using the [title, author, timestamp] notation. Every sentence MUST have a citation!'

    message = introduction

    question = f"\n\nQuestion: {question}"

    references = ""

    for i, result in enumerate(results):
        result = result + "\n\n"
        if (
            num_tokens(message + result + question, model=model)
            > token_budget
        ):
            break
        else:
            message += result
            start_timestamp, end_timestamp, iframe = refencify(result)
            references += f"### Segment {i+1} ({start_timestamp} - {end_timestamp}):\n" + iframe

    # Remove the last extra two newlines
    message = message[:-2]

    references = "Segments that might have been used to answer your question: (If you specified more segments than shown here, consider increasing your token budget)\n\n" + references

    return message + question, references

def generate_answer(question, model, token_budget, temperature):
    
    message, references = form_query(question, model, token_budget)

    messages = [
        {"role": "system", "content": "You answer questions about YouTube videos."},
        {"role": "user", "content": message},
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    response_message = response["choices"][0]["message"]["content"]

    return response_message, references

def add_to_dict(title, url):
    global title_counter

    if title not in titles_to_urls:
        # This is the first occurrence of this title
        titles_to_urls[title] = url
        return title
    else:
        # This title has already been seen, so we need to add a number suffix to it
        # First, check if we've already seen this title before
        if title in title_counter:
            # If we have, increment the counter
            title_counter[title] += 1
        else:
            # If we haven't, start the counter at 1
            title_counter[title] = 1
        
        # Add the suffix to the title
        new_title = f"{title} ({title_counter[title]})"
        
        # Add the new title to the dictionary
        titles_to_urls[new_title] = url
        return new_title

def search_youtube(question, n_videos):
    videosSearch = VideosSearch(question, limit = n_videos)
    urls = ["https://www.youtube.com/watch?v=" + video["id"] for video in videosSearch.result()["result"]]
    print(urls)
    return urls

In [19]:
def main(openAI_key, question, n_videos, urls_text, split_by_topic, segment_length, n_neighbours, model, token_budget, temperature):

    print(question)
    print(urls_text)

    set_openai_key(openAI_key)

    if urls_text == "":
        urls = search_youtube(question, n_videos)
    else:
        urls = list(set(urls_text.split("\n")))

    global titles_to_urls
    titles_to_urls = {}

    segments = []

    for url in urls:

        if "youtu.be" in url:
            url = url.replace("youtu.be/", "youtube.com/watch?v=")

        res = get_youtube_data(url)

        if not res:
            continue

        df, title, author = res
        
        title = add_to_dict(title, url)

        video_segments = get_segments(df, title, author, split_by_topic, segment_length)

        segments.extend(video_segments)
    
    if segments == []:
        return "Something wrong happened! Try specifying the YouTube videos or changing the query.", ""

    print("Segments generated successfully!")

    if fit_searcher(segments, n_neighbours):
        print("Searcher fit successfully!")
        answer, references = generate_answer(question, model, token_budget, temperature)

    print(answer)

    return answer, references

In [20]:
nltk.download('stopwords')
tt = TextTilingTokenizer()
searcher = SemanticSearch()

# Initialize a counter for duplicate titles
title_counter = {}

# One to one mapping from titles to urls
titles_to_urls = {}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
main(openAI_key="env", 
    question="What it does after extracting only the code", n_videos=5, 
    urls_text="https://youtu.be/l_r_b38VXmQ", 
    split_by_topic=False, 
    segment_length=200, 
    n_neighbours=5, 
    model="gpt-3.5-turbo", 
    token_budget=2000, 
    temperature=0)

What it does after extracting only the code
https://youtu.be/l_r_b38VXmQ
Segments generated successfully!
Fitting with n=5...
Searcher fit successfully!
Segment from 'GPT-4 Auto Coder 2 reads docs from URL links and Auto Debugs' by echohive
Timestamp: (12:01, 13:12)

exact same thing we tried at first right after this neural network demonstration so look so it's finished it and ran it there we go and test input I have successfully created synthetic data set and it's given its final answer yeah it's pretty crazy so look let's just grab our instructions without any links and give it to gpt4 and see how far it goes so you didn't even know what to do but I have added return of python script for this let's see what it does now as we see here we got some code loading of the text is fine but there's nothing about link chain it just simply doesn't know what Lang chain is it doesn't know what a QA chain is it's just trying to come up with something cool as far as it knows what's best but this i

('I could not find an answer.',
 'Segments that might have been used to answer your question: (If you specified more segments than shown here, consider increasing your token budget)\n\n### Segment 1 (12:01 -  13:12):\n<iframe\n    width="400"\n    height="240"\n    src="https://youtube.com/embed/l_r_b38VXmQ?start=721&end=792&controls=0"\n    frameborder="0"\n    allow="accelerometer; autoplay; modestbranding; encrypted-media; gyroscope; picture-in-picture"\n    allowfullscreen\n    >\n    </iframe>\n\n### Segment 2 (07:31 -  08:38):\n<iframe\n    width="400"\n    height="240"\n    src="https://youtube.com/embed/l_r_b38VXmQ?start=451&end=518&controls=0"\n    frameborder="0"\n    allow="accelerometer; autoplay; modestbranding; encrypted-media; gyroscope; picture-in-picture"\n    allowfullscreen\n    >\n    </iframe>\n\n### Segment 3 (08:38 -  09:46):\n<iframe\n    width="400"\n    height="240"\n    src="https://youtube.com/embed/l_r_b38VXmQ?start=518&end=586&controls=0"\n    frameborder=